In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

# 11.4 시간대 다루기

In [2]:
# 파이썬에서 시간대 정보는 전 세계의 시간대 정보를 모아둔 올슨 데이터베이스를 담고 있는 서드파티 라이브러리인 pytz에서 얻어온다.
# 이는 특히 역사적인 데이터를 다룰 때 중요한데 DST 날짜(그리고 심지어는 UTC 오프셋 마저)는 지역 정부의 변덕에 따라 여러 차례 변경되었기 때문이다. 
# 미국에서는 1900년부터 DST 시간이 수차례 변경되었다.
# 참고로 시계열을 다루는 많은 사용자는 현재 국제표준이며 그리니치 표준시를 계승하는 국제표준시(UTC)를 선택한다.

In [3]:
# pytz 라이브러리에 대한 자세한 내용은 라이브러리의 문서를 살펴봐야한다. 
# pandas는 pytz의 기능을 사용하고 있으므로 시간대 이름 외에 API의 다른 부분은 무시해도 상관없다. 
# 시간대 이름은 문서와 파이썬 셸에서 직접 확인할 수 있다.

In [4]:
import pytz

In [5]:
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [6]:
# pytz에서 시간대 객체를 얻으려면 pytz.timezone을 사용하면 된다.

In [7]:
tz = pytz.timezone("America/New_York")

In [8]:
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

In [9]:
# pandas의 메서드에서는 시간대 이름이나 객체를 모두 사용할 수 있지만 시간대 이름을 사용하기 권장한다. 

# 11.4.1 시간대 지역화와 변환

In [10]:
# 기본적으로 pandas에서 시계열은 시간대를 엄격히 다루지 않는다. 다음 시계열을 살펴보자. 

In [11]:
rng = pd.date_range("3/9'2012 9:30", periods=6, freq="D")

In [12]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [13]:
ts

2012-03-09 09:30:00   -0.204708
2012-03-10 09:30:00    0.478943
2012-03-11 09:30:00   -0.519439
2012-03-12 09:30:00   -0.555730
2012-03-13 09:30:00    1.965781
2012-03-14 09:30:00    1.393406
Freq: D, dtype: float64

In [14]:
# 색인의 필드는 None이다.

In [15]:
print(ts.index.tz)

None


In [16]:
# 시간대를 지정해서 날짜 범위를 생성할 수 있다.

In [17]:
pd.date_range("3/9/2012 9:30", periods=10, freq="D", tz="UTC")

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [19]:
# 지역화 시간으로의 변환은 tz_localize 메서드로 처리할 수 있다.

In [20]:
ts

2012-03-09 09:30:00   -0.204708
2012-03-10 09:30:00    0.478943
2012-03-11 09:30:00   -0.519439
2012-03-12 09:30:00   -0.555730
2012-03-13 09:30:00    1.965781
2012-03-14 09:30:00    1.393406
Freq: D, dtype: float64

In [21]:
ts_utc = ts.tz_localize("UTC")

In [22]:
ts_utc

2012-03-09 09:30:00+00:00   -0.204708
2012-03-10 09:30:00+00:00    0.478943
2012-03-11 09:30:00+00:00   -0.519439
2012-03-12 09:30:00+00:00   -0.555730
2012-03-13 09:30:00+00:00    1.965781
2012-03-14 09:30:00+00:00    1.393406
Freq: D, dtype: float64

In [23]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [24]:
# 시계열이 특정 시간대로 지역화되고 나면 tz_convert를 이용해서 다른 시간대로 변환 가능하다.

In [25]:
ts_utc.tz_convert("America/New_York")

2012-03-09 04:30:00-05:00   -0.204708
2012-03-10 04:30:00-05:00    0.478943
2012-03-11 05:30:00-04:00   -0.519439
2012-03-12 05:30:00-04:00   -0.555730
2012-03-13 05:30:00-04:00    1.965781
2012-03-14 05:30:00-04:00    1.393406
Freq: D, dtype: float64

In [26]:
# 위 시계열의 경우에는 America/New_York 시간대에서 일광절약시간을 사용하고 있는데, 동부 표준시(EST)로 맞춘 다음 UTC 혹은 베를린 시간으로 변환할 수 있다.

In [27]:
ts_eastern = ts.tz_localize("America/New_York")

In [28]:
ts_eastern.tz_convert("UTC")

2012-03-09 14:30:00+00:00   -0.204708
2012-03-10 14:30:00+00:00    0.478943
2012-03-11 13:30:00+00:00   -0.519439
2012-03-12 13:30:00+00:00   -0.555730
2012-03-13 13:30:00+00:00    1.965781
2012-03-14 13:30:00+00:00    1.393406
dtype: float64

In [29]:
ts_eastern.tz_convert("Europe/Berlin")

2012-03-09 15:30:00+01:00   -0.204708
2012-03-10 15:30:00+01:00    0.478943
2012-03-11 14:30:00+01:00   -0.519439
2012-03-12 14:30:00+01:00   -0.555730
2012-03-13 14:30:00+01:00    1.965781
2012-03-14 14:30:00+01:00    1.393406
dtype: float64

In [30]:
# tz_localize와 tz_convert는 모두 DatetimeIndex의 인스턴스 메서드다.

In [31]:
ts.index.tz_localize("Asia/Shanghai")

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq=None)

- CAUTION! 타임스탬프를 특정 시간대로 지역화하면 일광절약시간에 의한 모호하거나 존재하지 않는 시간을 체크한다.

# 11.4.2 시간대를 고려해서 Timestamp 객체 다루기

In [32]:
# 시계열이나 날짜 범위와 비슷하게 개별 Timestamp 객체도 시간대를 고려한 형태로 변환이 가능하다.

In [33]:
stamp = pd.Timestamp("2011-03-12 04:00")

In [34]:
stamp_utc = stamp.tz_localize("utc")

In [35]:
stamp_utc.tz_convert("America/New_York")

Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

In [36]:
# Timestamp 객체를 생성할 때 시간대를 직접 넘겨주는 것도 가능하다.

In [37]:
stamp_moscow = pd.Timestamp("2011-03-12 04:00", tz="Europe/Moscow")

In [38]:
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

In [39]:
# 시간대를 고려한 Timestamp 객체는 내부적으로 UTC 타임스탬프 값을 유닉스 에포크(1970년 1월 1일)부터 현재까지의 나노초로 저장하고 있다.
# 이 UTC 값은 시간대 변환 과정에서 변하지 않고 유지된다.

In [40]:
stamp_utc.value

1299902400000000000

In [41]:
stamp_utc.tz_convert("America/New_York").value

1299902400000000000

In [42]:
# pandas의 DateOffset 객체를 이용해서 시간 연산을 수행할 때는 가능하다면 일광절약시간을 고려한다.
# DST로 전환되기 직전의 타임스탬프에 대한 예제를 살펴보자.
# 먼저 DST 시행 30분 전의 Timestamp를 생성하자.

In [43]:
from pandas.tseries.offsets import Hour

In [44]:
stamp = pd.Timestamp("2012-03-12 01:30", tz="US/Eastern")

In [45]:
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [46]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

In [47]:
# 그리고 DST 시행 90분 전의 Timestamp를 생성하자.

In [48]:
stamp = pd.Timestamp("2012-11-04 00:30", tz="US/Eastern")

In [49]:
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [50]:
stamp + 2 * Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

# 11.4.3 다른 시간대 간의 연산

In [51]:
# 서로 다른 시간대를 갖는 두 시계열이 하나로 합쳐지면 결과는 UTC가 된다. 
# 타임스탬프는 내부적으로 UTC로 저장되므로 추가적인 변환이 불필요한 명료한 연산이다. 

In [59]:
rng = pd.date_range("3/7/2012 9:30", periods=10, freq="B")

In [60]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [61]:
ts

2012-03-07 09:30:00   -2.001637
2012-03-08 09:30:00   -0.371843
2012-03-09 09:30:00    1.669025
2012-03-12 09:30:00   -0.438570
2012-03-13 09:30:00   -0.539741
2012-03-14 09:30:00    0.476985
2012-03-15 09:30:00    3.248944
2012-03-16 09:30:00   -1.021228
2012-03-19 09:30:00   -0.577087
2012-03-20 09:30:00    0.124121
Freq: B, dtype: float64

In [62]:
ts1 = ts[:7].tz_localize("Europe/London")

In [63]:
ts2 = ts1[2:].tz_convert("Europe/Moscow")

In [64]:
result = ts1 + ts2

In [65]:
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)